<h1>Korean Text classification with KoBERT</h1>

## **Summary of the project**

In Korea, even though there are many research conducted being conducted on voice phishing, it remains a real case problem that technology such as artificial intelligence can tackle. Through a previous project conducted, we created a dataset containing phone call conversation transcripts and general conversation text data. This voice phishing dataset has two different class which are **voice phishing** (represented as "1") and **non-voice phishing** (represented as "0").

Using this dataset with state-of-the-art (SOTA) pre-trained word embedding [KoBERT](https://github.com/SKTBrain/KoBERT), we will perform NLP task such as text classification to build binary classification models.

## **Aim of the project**
In this project, we aim to build binary classification models capable to determine whether the inputted Korean conversation text is voice phishing ("1") or non-voice phishing ("0") related text.

The API used are Tensorflow for BERT model and Pytorch for KoBERT model.

## **Desired outputs of the project**
From the trained models, we expect to achieve great classification performance on this voice phishing dataset such as the model tells us if a conversation is harmful or not harmful.
At the end of this project, we will look at the accuracy of the model on the test set.

# Training the binary classification model with KoBERT

In [1]:
!nvidia-smi

Wed Apr 26 04:32:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installing the common needed packages

In [2]:
# dowload and install KoBERT as a python package
# this commande will install the requirted package at the same time
  # gluonnlp >= 0.6.0
  # mxnet >= 1.4.0
  # onnxruntime >= 0.3.0
  # sentencepiece >= 0.1.6
  # torch >= 1.7.0
  # transformers >= 4.8.1

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-uo2kz3_r
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-uo2kz3_r
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 45.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 109.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 2.0 MB/s eta 0:0

## Import all the needed libraries

In [3]:
## importing the required packages
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

from sklearn.model_selection import train_test_split

In [4]:
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
# from keras.models import Sequential
from sklearn.utils import shuffle

In [5]:
## importing KoBERT functions
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
## import transformers functions
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
## Configure the GPU  device
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

## Importing the dataset

In [8]:
"""
Since we are using Colab, we will provide a test to check if environment is 
colab or not so that the data can also be imported in case this jupyter file is 
ran on local machine and not on colab
"""

if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  ## mount the google drive
  from google.colab import drive
  drive.mount('drive')
  # move to the directory where dataset is saved
  %cd drive/My\ Drive/Colab\ Notebooks/
else:
  print('Not running on CoLab')

Running on CoLab
Mounted at drive
/content/drive/My Drive/Colab Notebooks


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# 기존 연구의 데이터셋
dataset = pd.read_csv('/content/drive/MyDrive/KoBERT/KorCCViD_v1.3_fullcleansed.csv', encoding='utf8').sample(frac=1.0)
dataset.sample(n=15)

,Transcript,Label
854,근까 그렇게 느낀 근데 막상 사귀 우리 얘기 얘기 니까 그제서야 갑자기 자기 그게 ...,0
418,계신 동생 라고 고요 으시 굳이 먼저 말씀 셔도 고요 그러 지금 올려 드리 오전 1...,1
421,근데 작년 어떤 분위기 냐면 그냥 면서 사람 끼리 마음 편하 니까 놀리 대요 망토 ...,0
397,그거 필름 메이스 커스 올라오 올리 대학생 과제 배우 필요 다고 그래서 거기 저희 ...,0
543,그래 그거 면서 되게 어떤 그런 웃음 포인트 라던지 되게 잡아내 되게 연극 라는 생...,0
1203,자긴 저흰 자기 그런 하나 오히려 체벌 금지 여서 선생 체벌 학부모 난리 그랬 아직...,0
786,사이트 라고 혹시 계십니까 자주 신다면 습니까 사용 사이트 계시 물품 등록 판단 명...,1
1191,왜냐하면 그냥 모의고사 생각 면은 그냥 건대 정도 건동 라인 근데 여기 완전 망해 ...,0
154,여름 겨울 사람 아니 잖아 근데 아무래도 요즘 신경 부분 아마 체력 감소 라고 생각...,0
355,주세요 다름 아니 그걸 거든 그분 하고 실행 어디 빨리,1


In [12]:
# 본 연구의 데이터 셋
dataset_val = pd.read_csv('/content/drive/MyDrive/KoBERT/val_test_my_fullcleansed.csv', encoding='utf-8')
dataset_val.sample(n=15)

,transcript,label
29,고객 십일월 목요일 오후 지금 신애 저축 에서 전화 드렸 구요 금융 조사 부장 라고...,1
26,신용 대출 얼마나 신용 대출 고객 조건 따라 다릅니다 직업 무엇 회사원 입니다 대출...,0
14,일단 백만 34 나오 어요 34 저희 상환 방식 원금 같이 납부 납부 방법 중도 상...,1
43,그래서 대출금 나가 고객 금융 관련 해서 전산 원래 통합 으로 심사 부분 그런 부분...,1
63,아파트 담보 대출 혹시 추가 대출 한가요 그러 군요 추가 대출 상담 위해 참가자 본...,0
45,근데 어떻게 라는 얘기 고객 으로 입금 내역 백만 으셔야 돼요 지금 어요 그거 고객...,1
64,정해진 저희 고객 한테 저희 회사 자금 회수 다시 회사 입금 처리 외부 경리 신다고...,1
65,에서 송금 할까요 국내 또는 국외 어느 으로 송금 건가요 국내 합니다 고객 그럼 으...,0
1,대출 원리금 상환 어떻게 하나 고객 계좌 됩니다 계좌 다구요 고객 대출 마다 계좌 ...,0
11,본점 으로 서류 보내 드릴 에요 네네 그러면 고객 대출금 승인 오늘 저희 으로 본점...,1


## Data transformation and splitting

In [13]:
## transform preceding study train set and val set into tsv file to usedd into KoBERT
# train_tsv = nlp.data.TSVDataset('KorCCViD_v1.3_fullcleansed.csv')
# train_tsv = nlp.data.TSVDataset('KorCCViD_v1.3_fullcleansed.csv')

dataset_tsv = []
for text, label in zip(dataset['Transcript'], dataset['Label']):
    data = []
    data.append(text)
    data.append(str(label))

    dataset_tsv.append(data)

In [14]:
dataset_tsv[:5]

[['그래서 일찍 연락 드린 건데 맛있 내용 정리 정도 사용 설명 드리 자면 옥션 마켓 인터넷 인터넷 에서 저렴 판매 한다고 올려 전화 아서 어요 통장 농협 통장 고요 그분 처음 26 26 고소 고발 상황 에요 하지만 부분 대해서 해서 저녁 셔야죠 저희 에서 진행 부분 고요 셔서 법원 에서',
  '1'],
 ['챔피언 일단 그니까 처음 나왔 나온 캐릭터 그리고 마다 게임 출시 그래서 시즌 시즌 패스 시즌 패스 시즌 패스 으로 구성 패스 나오 모든 오퍼 모든 캐릭터 기본 으로 그런 스킨',
  '0'],
 ['220 처리 라고 나와 그거 으시 지금 처리 신가요 220 라고 얘기 50 라고 50 상담원 아니 50 으로 구요 죄송 한데요 여기 취소 세요 취소 다구요 취소 고객 납부 까지 주일 정도 소요 세요 상관없 으신 세요 습니다 드릴게요 그럼 취소 어떻게 에요 저희 에서 취소 처리 올리 돼요 주일 다음 월요일 정도 해서 환급 들어가 겁니다 환급 처리 정확 된다는 어떻 확인 본사 전화 저희 연락 근데 삭제 해도 건가요 근데 서부터 에서 다른 핸드폰 으로 전화 더니 다고 던데 본인 인증 무슨 에요 핸드폰 다른 핸드폰 으로 전화 니까 본인 핸드폰 으로 전화 셔야 확인 그리고 에도 전화 다고 던데 본인 핸드폰 으로 전화 확인 세요 금융 여보세요 보이스 피싱 으시 아닙니다 심사 처리 정확히 언제 에요 다음 월요일 들어가 에요 정확 들어오 나요 습니다 보이스 피싱 으시 잖아요 아닙니다 어떻게 확인 에요 본사 에서 전화 본사 에서 본인 인증 된다는 경찰 신고 해도 돼요 그건 상관 어요 습니다 으니까 신고 드릴게요',
  '1'],
 ['통장 경우 해당 금융 데려 결과 확실히 위조 통장 아니 통장 확인 고요 우리 하나 인데 해서 발견 니까 그래도 광명 발급 경기도 광명시 부분 에요 본인 직접 발급 으신 아니 라는 최근 지갑',
  '1'],
 ['빨리 취업 방학 으니까 그래서 진짜 어떻게 만약 목표 540 550 지만 그거 더라도 일단 유럽 무조건 그냥 그런 다른 사람 니까

In [15]:
## transform our test set into test file to usedd into KoBERT

dataset_test = []
for text, label in zip(dataset_val['transcript'], dataset_val['label']):
    data = []
    data.append(text)
    data.append(str(label))

    dataset_test.append(data)

In [16]:
dataset_test[:5]

[['여보세요 여보세요 고객 금방 통화 담당자 김민정 인데요 그게 네이버 에서 혹시 바탕 화면 동그랗 모양 인터 으로 들어가 실까요 고객 구글 말씀 에요 아니 그냥 인터넷 지구 모양 인터넷 거기 들어가 구요 주소창 나오 잖아요 고객 주소창 주소창 지금 영문 으로 잖아요 그게 영문 으로 주소 지워 구요 지우 하나 남김없이 지우 상태 에서 지우 상태 에서 불러 드릴게요 그리고 그리고 버튼 눌러 세요 고객 여기 들어가 인증 번호 교체 해야 때문 본인 인증 화면 넘어가 나요 고객 여기 잠시 연결 아직 진행 어서 혹시 인터넷 조금 어서 그런 아닐까요 지금 넘어가 넘어간다 고요 인터넷 으면은 그럴 일반 홈페이지 나요 아이피 왜냐면 본인 인증 화면 에서 직접 셔야 돼요 고객 본인 본인 우편물 저희 직접 불러들이 거든요 그리고 저희 홈페이지 다운 으셔야 지금 인터넷 에서 연결 면은 아직 넘어간다는 말씀 세요 그래요 고객 그러 다른 부분 본인 인증 구요 만약 부분 넘어가 인증 번호 다시 바꿔서 말씀 드릴게요 습니다 고객 네이버 에서 다시 클릭 세요 객님 확인 한번 확인 세요 시간 연락 어요 저희 한테 내일 아침 해야 그래요 12 고객 담당자 김민정 고요 고객 전화 드리 전화 세요',
  '1'],
 ['대출 원리금 상환 어떻게 하나 고객 계좌 됩니다 계좌 다구요 고객 대출 마다 계좌 부여 습니다 모르 계좌 어떻게 센터 에서 상담원 통해 확인 습니다 센터 번호 어떻게 나요 지금 번호 나요 대출 관련 부서 번호 다릅니다 에서 상담원 연결 확인 습니다 상담원 에게 계좌 확인 나요 확인 납입 됩니다 인터넷 뱅킹 으면 인터넷 으로 확인 합니다 인터넷 뱅킹 어서요 상환 수수료 나요 계좌 등록 어서 수수료 발생 습니다 대출 인데 계좌 마다 각각 보내 나요 고객 대출 좌도 입니다 각각 보내 됩니다 만약 계좌 출금액 모두 송금 어떻게 나요 그러면 입금 계좌 연결 대출 상환 처리 나머지 금액 고객 다시 환급 됩니다 계좌 다른 방법 으로 상환 나요 죄송 합니다',
  '0'],
 ['항상 채무 

In [17]:
train_set, val_set = train_test_split(dataset_tsv, 
                               test_size=0.2, 
                               random_state=42, 
                               shuffle=True)

test_set = shuffle(dataset_test, random_state = 42)

print(f"Numbers of train instances by class: {len(train_set)}")
print(f"Numbers of val instances by class: {len(val_set)}")
print(f"Numbers of test instances by class: {len(test_set)}")



Numbers of train instances by class: 974
Numbers of val instances by class: 244
Numbers of test instances by class: 70


In [18]:
for i in range(10,20):
    print(val_set[i])

['지금 본인 명의 지금 도영 지금 대포통장 금융 사기 범죄 지금 여유 습니다 근데 부분 때문 저희 확인 어서 지금 전화 연락 드린 부분 하나 지금 농협 지금 대포통장 으로 지금 어요 지금 부분 대해서 조금 조금 말씀 드리 습니다 본인 께서 김현우 라고 계시 인가요 왜냐면은 저희 우유 사기 공부 습니다 에서 대장 신용 카드 보안 카드 대포통장 제값 에서 아까 말씀 드렸 명의 하나 통장 농협 통장 통장 발견 습니다 혹시 통장 개설 여부 대해서 혹시 내용 으신 개소리 2015 11 고요 경기도 철산동 에서 개설 습니다 경기도 광명시 철산동 에서 사진 으신 대가 선고 판매 거나 그러 부분 전혀 으신 사이 핸드폰 지갑 신분증 운전 면허증 혹시 개인 정보 유출 그런 모종 그런 분실 거나 도난 당한 으신 저녁 고요 일단 저희 경우 저희 지금 사건 조사 진행 부분 고요 왼쪽 으로 인터넷 그런 부분 에서 유출 그거 아닌 또는 지금 현재 김현우 식당 68 예전 정세 사람 입니다 부분 어서 지금 지금 나중 부분 고요 착각 부분 현재 사건 연휴 100 정도 지금 사건 연료 지금 거기 에서 거기 90 에서 100 그분 정도 금전 판매 금액 인식 택시 처럼 전혀 모르 사람 일단 통장 어떻 어떤 으로 복귀 해서 어떻게 사용 전혀 모르 계신 계세요 그런 지금 분별 거기 에서 아니 신분 필요 자격증 으시 라고 지금 전화 유선 으로 전화 연락 드리 겁니다 지금 본인 명의 만들 통장 아까 말씀 드렸 농협 통장 으로 인해서 정도 금액 으니까 거기 에서 피해 사람 지금 통장 주인 식이 때문 고소 고발 상태 에서 그래서 지금 바로 연락 드린 겁니다 지금 지금 연락 드리 지금 일단 사건 번호 지금 혹시 메모 신가요 혹시 지금 한테 계세요 아니면 직장 근무 세요 메모 부탁 드리 습니다 보이스 피싱 그러 한다고 순천 택시 불러 드릴게요 송금 번호 2015 날씨 한글 번호 지금 불렀 더니 번호 으로 나와 사건 번호 고요이 경우 저기 검사 다시 한번 확인 시켜 드릴 겁니다 다시 확인 시켜 드리 직접 으로 

In [19]:
val_set[12][0]

'그리고 거기 설명 아니 자랑 하나 자기 데려갔 여행객 터졌 다는 그래서 12 거기 그리고 그렇게 상금 12 으면은 거기 상금 나라 에서 라고 영주 지급 준다는 그래서 사람 여행 한국 돌아오 그냥 거기 에서 마음먹 었었'

In [20]:
for i in range(10,20):
    print(test_set[i])

['인터넷 뱅킹 에서 그런 고객 혹시 장기 사용 신가요 그게 인터넷 뱅킹 통한 거래 경우 금융 서비스 중지 합니다 그런가 봐요 사용 어요 어떻게 해야 다시 사용 등록 방법 으로 습니다 인터넷 뱅킹 으로 알려 세요 아이디 비밀 번호 보안 카드 계좌 번호 비밀 번호 모두 입력 처리 합니다 영업 으로 합니다 필요 서류 나요 신분증 지참 전자 금융 신고 서류 작성 해제 합니다 전화 으로 나요 전화 으로 신분 확인 불가 합니다 만약 인터넷 뱅킹 사용 비밀 번호 버리 공인 증서 등록 으면 그걸로 대체 합니다 보안 카드 어야 필요 합니다 감사', '0']
['내역 확인 어디 확인 인터넷 뱅킹 사용 계신 인터넷 모바일 통해 확인 어요 인터넷 뱅킹 로그인 메뉴 하단 내역 확인 됩니다 모바일 역시 동일 확인 습니다 인터넷 에서 보낸 모바일 에서 확인 나요 어떤 방법 으로 셔도 내역 에서 확인 습니다 사람 계좌 번호 확인 수취인 성함 계좌 번호 모두 확인 합니다 매월 모임 자동 메뉴 하단 자동 설정 에서 체일 주기 금액 설정 습니다 타행 수수료 발생 나요 타행 건당 수수료 발생 됩니다 우수 고객 인데 수수료 나요 확인 습니다 고객 골드 고객 으로 10 면제 됩니다 그럼 10 초과 어떻게 나요 10 초과 20 까지 수수료 50 부과 됩니다 금액 나요 금액 상관없이 회당 수수료 부과 됩니다 고객 등급 계속 유지 건가요 분기 마다 고객 거래 실적 따라 조정', '0']
['대출금 연말 정산 서류 발급 방법 알려 세요 대출금 연말 정산 서류 발급 센터 인터넷 통한 신청 영업 공과금 인수 납기 통한 발급 습니다 센터 통해서 신청 어요 발급 수수료 면제 우편 팩스 습니다 그럼 시간 걸리 네요 우편 송시 대략 정도 소요 일정 기일 여유 신청 바랍니다 너무 오래 걸려서 방법 아요 영업 방문 즉시 발급 신데 영업 방문 어떠세요 그러면 요새 코로나 때문 에서 어요 그러 인터넷 통한 발급 영업 공과금 인수 납기 에서 신청 방법 추천 드릴게요 영업 공과 금수 납기 에서 발급 으면 수수료 어요 발급 수

In [21]:
test_set[12][0]

'대출금 연말 정산 서류 발급 방법 알려 세요 대출금 연말 정산 서류 발급 센터 인터넷 통한 신청 영업 공과금 인수 납기 통한 발급 습니다 센터 통해서 신청 어요 발급 수수료 면제 우편 팩스 습니다 그럼 시간 걸리 네요 우편 송시 대략 정도 소요 일정 기일 여유 신청 바랍니다 너무 오래 걸려서 방법 아요 영업 방문 즉시 발급 신데 영업 방문 어떠세요 그러면 요새 코로나 때문 에서 어요 그러 인터넷 통한 발급 영업 공과금 인수 납기 에서 신청 방법 추천 드릴게요 영업 공과 금수 납기 에서 발급 으면 수수료 어요 발급 수수료 면제 입니다 나가 귀찮 으니까 그냥 인터넷 발급 방법 알려 세요 지금 바로 접속 해서 발급 으실 당사 홈페이지 접속 셔서 뱅킹 관리 눌러 세요 어요 증명 발급 눌러 연말 정산 증명서 클릭 주택 자금 대출 상환 증명서 발급 으시 됩니다 네요 감사 합니다 도움 다니 다행'

## Prepare the data as input for the KoBERT model
According tot he documentation the class BERTDataset is to be used to perform in the background the following tasks.
- Tokenization
- Numericalization (encoding string to integer)
- Padding
- etc



In [22]:
# Definition of BERTDataset class (mandatory)
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [23]:
# Setting the hyperparameters
max_len = 64 # The maximum sequence length that this model might ever be used with. 
             # Typically set this to something large just in case (e.g., 512 or 1024 or 2048).
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10   # only parameter changed from 5 to 10 compared to the documentation
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5  # 4e-5

In [24]:
# Perform the prearation task of the data using class defined above
bertmodel, vocab = get_pytorch_kobert_model() # calling the bert model and the vocabulary

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

train_set = BERTDataset(train_set, 0, 1, tok, max_len, True, False)
val_set = BERTDataset(val_set, 0, 1, tok, max_len, True, False)
test_set = BERTDataset(test_set, 0, 1, tok, max_len, True, False)

using cached model. /content/drive/MyDrive/Colab Notebooks/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/Colab Notebooks/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/drive/MyDrive/Colab Notebooks/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [25]:
print(val_set[12][0])

[   2 1210  862 5561 2773 3097 3894 4928 3890 1706 6060 5348 3321 5370
 4728 7248 1562 5760 1195  539  862 5561 1210 1204 2658 5550  539  517
 7083 7086  862 5561 2658 5550 1376  517 6903  517 6004 3376 7276 4302
 4248 5760 1195 2584 3321 4958 1736 1189  862 5561  517 6903 1917 6183
  517 6885 6885    3    1    1    1    1]


In [26]:
print(test_set[12][0])

[   2 1666 5550 3345 4092 6516 2721 2239 2270 3169  517 6586 1666 5550
 3345 4092 6516 2721 2239  517 6593 3794 4754 3030 3382 1023 5468 5550
 3777  517 5669 5561 4754 2239  517 6701  517 6593 4756 6553 3030  517
 6857 2239 2887 2033 3498 7720  517 7700 6664  517 6701 1185 6043 2962
  889 6122  517 5703 3498 7720 2869    3]


In [27]:
tokenizer

'/content/drive/MyDrive/Colab Notebooks/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece'

In [28]:
tok

In [29]:
# verifying the transformation
train_set[1]

(array([   2, 3298, 6364, 6586, 3298, 6364, 6586, 2207, 5345,  517, 6701,
        1185, 6043,  517, 7930, 6705, 4304,  567, 3093, 6749, 5771, 1231,
        7147, 4252, 7096,  517, 7076, 5591, 3097, 6999, 3129, 5130, 3010,
        5330,  517, 7076, 5591, 2302, 6134, 2432, 3924, 3343, 1201, 4299,
        3343, 1633, 7728, 7636, 7178, 4809,  881, 7318, 3097,  847, 4103,
        3593, 2034, 7953,  993, 6542, 4915, 5130, 3112,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [30]:
# verifying the transformation
test_set[1]

(array([   2, 3298, 6364, 6586, 3298, 6364, 6586,  994, 1235, 6305, 4758,
        1610, 7147, 1320, 7227,  517, 7120, 6999, 1185, 5400, 1470,  517,
        6903,  517, 7930, 6705, 2186, 7597, 5114, 1741, 5538,    0, 2063,
        3793,  517, 7078, 1805, 3036, 5591, 6999,  994, 1117, 1961,  517,
        6896, 6999, 3097, 1189, 3794, 4298, 2063, 3794,  862, 5561, 1805,
        1115, 6999, 4213, 6607, 7402, 1388,  517, 7173,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [31]:
# creating torch-type datasets
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, num_workers=5)
val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## Creation of the KoBERT learing model

In [32]:
# This class is from the GitHub repository and the documentation
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   # since we are in binary classification we set the value 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [33]:
import torch
# torch.cuda.empty_cache()

In [34]:
# creation of the model
model = BERTClassifier(bertmodel,  dr_rate=0.4).to(device)

In [35]:
%%time
print(model)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [36]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [37]:
# configuration f the optimizer and loss function
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [38]:
# define the function to compute the accury of the model
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [39]:
# model.summary()

In [40]:
def get_metrics(pred, label, threshold=0.5):
    pred = (pred > threshold).astype('float32')
    tp = ((pred == 1) & (label == 1)).sum()
    fp = ((pred == 1) & (label == 0)).sum()
    fn = ((pred == 0) & (label == 1)).sum()
    
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * recall * precision / (precision + recall)
    
    return {
        'recall': recall,
        'precision': precision,
        'f1': f1
    }

## Training the KoBERT model


In [41]:
%%time
from time import time
from timeit import default_timer as timer

# Training code from the github library
start_time = time()

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    # Training of the model with the train set
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    preds = []
    labels = []
    # evaluation of the model train on the test set
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        labe2 = label.cpu()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
        
        pred = out.detach()
        pred = F.softmax(pred)
        pred = pred[:, 1].cpu().numpy().tolist()
        preds += pred
        labels += label.cpu().numpy().tolist()
        
    preds = np.array(preds)
    labels = np.array(labels)
    metrics = get_metrics(preds, labels)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    # print('ACCURACY 2 = ', accuracy_score(out, label))
    print('Metrics: ', metrics)

run_time = time() - start_time

  3%|▎         | 1/31 [00:00<00:17,  1.76it/s]

epoch 1 batch id 1 loss 0.7364157438278198 train acc 0.53125


100%|██████████| 31/31 [00:11<00:00,  2.77it/s]

epoch 1 train acc 0.7883064516129032



100%|██████████| 8/8 [00:01<00:00,  7.37it/s]

...

epoch 10 batch id 1 loss 0.0006024321774020791 train acc 1.0


100%|██████████| 31/31 [00:12<00:00,  2.55it/s]

epoch 10 train acc 1.0



100%|██████████| 8/8 [00:01<00:00,  6.77it/s]

epoch 10 test acc 0.99609375
Metrics:  {'recall': 0.9921875, 'precision': 1.0, 'f1': 0.996078431372549}
CPU times: user 1min 8s, sys: 59.2 s, total: 2min 7s
Wall time: 2min 23s


In [42]:
run_time
#224.96386766433716
#0.9947

143.15894389152527

In [43]:
preds = []
labels = []
test_acc = 0.0
# evaluation of the model train on the test set
model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    labe2 = label.cpu()
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)

    pred = out.detach()
    pred = F.softmax(pred)
    pred = pred[:, 1].cpu().numpy().tolist()
    preds += pred
    labels += label.cpu().numpy().tolist()
    
preds = np.array(preds)
labels = np.array(labels)
metrics = get_metrics(preds, labels)
print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
# print('ACCURACY 2 = ', accuracy_score(out, label))
print('Metrics: ', metrics)

  0%|          | 0/3 [00:00<?, ?it/s]<ipython-input-43-1dd53a5817f9>:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = F.softmax(pred)
100%|██████████| 3/3 [00:00<00:00,  6.33it/s]

epoch 10 test acc 0.5
Metrics:  {'recall': 1.0, 'precision': 0.5, 'f1': 0.6666666666666666}
